# Benchmark EDUC

This notebook aims to benchmark the economic dispatch and unit commitment problem.

In [1]:
import warnings
import logging
import ams
import numpy as np
import pandas as pd

ams.config_logger(stream_level=50)

%run ../benchmarks.py
_ = get_tool_versions()

Last run time: 2025-02-02 00:25:51
Python: 3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:36:57) [Clang 15.0.7 ]

Tool        Version
----------  ------
ltbams      1.0.2 
andes       1.9.3 
cvxpy       1.6.0 
gurobipy    12.0.1
mosek       11.0.5
piqp        0.4.2 
pandapower  2.14.7
numba       0.60.0


In [2]:
ams.config_logger(stream_level=50)

In [3]:
cases = [
    './cases/pjm5bus_uced.xlsx',
    './cases/ieee14_uced.xlsx',
    './cases/ieee39_uced.xlsx',
    './cases/npcc_uced.xlsx',
    './cases/wecc_uced.xlsx',
]

In [4]:
solvers=['GUROBI', 'MOSEK']

In [5]:
# run individual case to ensure everything is working
sp = ams.load(cases[0],
              setup=True, no_output=True, default_config=True)
pre_time, sol = time_routine(sp, routine='ED', solvers=solvers)
print(pre_time)
print(sol)

Set parameter Username
Set parameter LicenseID to value 2617183
Academic license - for non-commercial use only - expires 2026-02-02
{'ams_mats': 0.0006, 'ams_parse': 0.0055, 'ams_eval': 0.0061, 'ams_final': 0.0001, 'ams_postinit': 0.0}
{'GUROBI': {'time': 0.0295, 'obj': np.float64(62.022999999999996)}, 'MOSEK': {'time': 0.0477, 'obj': np.float64(62.023000001038014)}}


/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


In [6]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logging.getLogger('pandapower').setLevel(logging.ERROR)

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

time_data_ed = np.zeros((n_iters, n_cases, len(cols_pre) + len(solvers)))
obj_data_ed = np.zeros((n_cases, len(solvers)))

time_data_uc = np.zeros((n_iters, n_cases, len(cols_pre) + len(solvers)))
obj_data_uc = np.zeros((n_cases, len(solvers)))

In [7]:
# Run ED
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    system = ams.load(case, setup=True, no_output=True, default_config=True)
    for n_iter in range(n_iters):
        pre_time, sol = time_routine(system, routine='ED',
                                     solvers=solvers,
                                     ignore_dpp=True)
        time_data_ed[n_iter, n_case, :len(pre_time)] = np.array(
            [i for i in pre_time.values()])
        time_data_ed[n_iter, n_case, len(pre_time):] = np.array(
            [sol[solver]['time'] for solver in solvers])
    obj = np.array([sol[solver]['obj'] for solver in solvers])
    obj_data_ed[n_case, :] = obj

Case: ./cases/pjm5bus_uced.xlsx
Case: ./cases/ieee14_uced.xlsx
Case: ./cases/ieee39_uced.xlsx
Case: ./cases/npcc_uced.xlsx
Case: ./cases/wecc_uced.xlsx


In [8]:
# Run UC
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    system = ams.load(case, setup=True, no_output=True, default_config=True)
    for n_iter in range(n_iters):
        pre_time, sol = time_routine(system, routine='UC',
                                     solvers=solvers,
                                     ignore_dpp=True)
        time_data_uc[n_iter, n_case, :len(pre_time)] = np.array(
            [i for i in pre_time.values()])
        time_data_uc[n_iter, n_case, len(pre_time):] = np.array(
            [sol[solver]['time'] for solver in solvers])
    obj = np.array([sol[solver]['obj'] for solver in solvers])
    obj_data_uc[n_case, :] = obj

Case: ./cases/pjm5bus_uced.xlsx
Case: ./cases/ieee14_uced.xlsx
Case: ./cases/ieee39_uced.xlsx
Case: ./cases/npcc_uced.xlsx
Case: ./cases/wecc_uced.xlsx


In [9]:
# Process and display the results
case_names = [case.split('/')[-1].split('.')[0] for case in cases]

time_ed = pd.DataFrame(columns=cols_pre + solvers, index=case_names)
for case in case_names:
    time_ed.loc[case] = time_data_ed[:, case_names.index(case), :].mean(axis=0)

time_uc = pd.DataFrame(columns=cols_pre + solvers, index=case_names)
for case in case_names:
    time_uc.loc[case] = time_data_uc[:, case_names.index(case), :].mean(axis=0)

# scale to ms
time_ed *= 1000
time_uc *= 1000

In [10]:
time_ed

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,GUROBI,MOSEK
pjm5bus_uced,0.5,3.34,5.26,0.18,0.0,19.91,16.14
ieee14_uced,0.6,3.38,5.25,0.18,0.0,26.29,24.41
ieee39_uced,0.7,3.4,5.78,0.18,0.0,37.98,44.76
npcc_uced,1.32,3.47,13.94,0.44,0.0,208.07,191.46
wecc_uced,1.4,3.41,14.23,0.35,0.0,140.49,151.25


In [11]:
time_uc

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,GUROBI,MOSEK
pjm5bus_uced,0.63,5.05,5.63,0.22,0.19,27.5,61.59
ieee14_uced,0.61,4.9,10.83,0.2,0.12,30.16,73.96
ieee39_uced,0.8,4.94,6.78,0.21,0.15,58.38,111.54
npcc_uced,1.29,4.86,12.7,0.42,0.17,324.19,541.98
wecc_uced,1.42,4.8,9.69,0.36,0.13,308.03,448.3


In [12]:
time_ed.to_csv('../results/results_ed.csv')
time_uc.to_csv('../results/results_uc.csv')